In [20]:
# pip install flask requests pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.6 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 508 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 14.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
This script demonstrates how to get an access token from the Spotify API using the client credentials flow.
"""
from config import CLIENT_ID, CLIENT_SECRET
import requests

# Define the URL and headers
url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

# Prepare the data payload
data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}

# Make the POST request
response = requests.post(url, headers=headers, data=data)

# Check if the request was successful and print the response
if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Access Token:", access_token)
else:
    print("Failed to get token:", response.status_code, response.text)


Access Token: BQAApqKWse3UysovlPEHI9fCQTIXsSYfe7fojsG6H7yA_TS8CV4v-EjmPcDZKZdVpHWj7D-kVg7DBYYHkWyrPDpUoCYdcSGBjh7CbdaKmLvJqPbeoYk


In [69]:
"""
Runs the Flask server to handle Spotify authorization and token retrieval.
Uses token to fetch user's tracks and audio features and add them to a DataFrame.
"""
import subprocess
import requests
import json
import time
import pandas as pd

# Step 1: Start your Flask server using subprocess
flask_process = subprocess.Popen(['python', 'spotify-login.py'])

# Step 2: Prompt the user to authorize the app
print("Please go to http://localhost:8888/login to authorize the app.")
print("After authorizing, press Enter to continue...")
input()  # Wait for user input after authorization

# Step 3: Load the access token from the saved JSON file
def get_access_token():
    try:
        with open('spotify_token.json', 'r') as token_file:
            tokens = json.load(token_file)
            return tokens.get('access_token')
    except FileNotFoundError:
        print("Token file not found. Ensure you have completed the authorization.")
        return None

# Fetch the access token
access_token = get_access_token()
if access_token:
    print("Access token obtained successfully.")
else:
    print("Failed to obtain access token.")

# Step 4: Function to get user's saved tracks with pagination
def get_all_saved_tracks(access_token):
    url = "https://api.spotify.com/v1/me/top/tracks"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    all_tracks = []
    offset = 0
    limit = 50  # Maximum limit allowed by Spotify API

    while True:
        params = {
            'type': 'tracks',
            'time_range': 'long_term',
            'limit': limit,
            'offset': offset
        }
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()
        items = data.get('items', [])
        if not items:
            break
        
        # Collect track information
        for i, track in enumerate(items):
            all_tracks.append({
                'track_id': track['id'],
                'track_name': track['name'],
                'artist': track['artists'][0]['name'],
                'explicit': track['explicit'],
                'popularity': track['popularity'],
                'album_name': track['album']['name']
            })
        
        # Increment offset for next batch
        offset += limit
        print(f"Fetched {len(all_tracks)} tracks so far...")

        # Avoid hitting rate limits
        time.sleep(0.1)

    return all_tracks

# Step 5: Function to get audio features for multiple tracks
def get_audio_features(access_token, track_ids):
    url = "https://api.spotify.com/v1/audio-features"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    audio_features = []

    # Fetch in batches of 100 (Spotify API limit)
    for i in range(0, len(track_ids), 100):
        batch_ids = track_ids[i:i + 100]
        params = {
            'ids': ','.join(batch_ids)
        }
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()
        items = data.get('audio_features', [])
        
        # Collect track information
        for i, track in enumerate(items):
            if track is None:
                continue
            audio_features.append({
                'track_id': track['id'],
                'duration_ms': track['duration_ms'], 
                'acousticness': track['acousticness'], 
                'danceability': track['danceability'], 
                'energy': track['energy'], 
                'instrumentalness': track['instrumentalness'], 
                'key': track['key'], 
                'liveness': track['liveness'], 
                'loudness': track['loudness'], 
                'mode': track['mode'], 
                'speechiness': track['speechiness'], 
                'tempo': track['tempo'], 
                'valence': track['valence']
            })
        
        # Avoid hitting rate limits
        time.sleep(0.1)
    
    return audio_features

# Step 6: Fetch all saved tracks and audio features, then combine into a DataFrame
if access_token:
    print("Fetching all saved tracks...")
    all_tracks = get_all_saved_tracks(access_token)
    
    if all_tracks:
        # Convert track data to a DataFrame
        df_tracks = pd.DataFrame(all_tracks)
        
        # Extract track IDs
        track_ids = df_tracks['track_id'].tolist()
        
        # Fetch audio features for all tracks
        print("Fetching audio features for tracks...")
        audio_features = get_audio_features(access_token, track_ids)
        df_audio_features = pd.DataFrame(audio_features)

        # Merge track data with audio features
        df = pd.merge(df_tracks, df_audio_features, left_on='track_id', right_on='track_id', how='inner')
        df['rank'] = range(1, len(df_audio_features) + 1)
        
        # Reorder columns for readability
        columns_order = ['rank', 'track_name', 'artist', 'album_name', 'duration_ms', 
                         'explicit', 'popularity', 'acousticness', 'danceability', 
                         'energy', 'instrumentalness', 'key', 'liveness', 
                         'loudness', 'mode', 'speechiness', 'tempo', 'valence']
        df = df[columns_order]

        # Save the DataFrame to a JSON file
        df.to_json('tracks.json', orient='records', lines=False)

        # Display the DataFrame
        print("All Tracks with Audio Features:")
        display(df)
    else:
        print("No tracks found or failed to fetch tracks.")
else:
    print("Access token is missing. Unable to proceed.")

# Step 7: Terminate the Flask server once done
flask_process.terminate()

Please go to http://localhost:8888/login to authorize the app.
After authorizing, press Enter to continue...
 * Serving Flask app 'spotify-login'
 * Debug mode: on


Address already in use
Port 8888 is in use by another program. Either identify and stop that program, or start the server with a different port.
127.0.0.1 - - [13/Nov/2024 18:09:23] "GET /login HTTP/1.1" 302 -
127.0.0.1 - - [13/Nov/2024 18:09:25] "GET /callback?code=AQD32HE1X_flxDnQgLtHDajQjSyODnVd8juzP5cR4ik5vpxxRQmgzNctpy3abDPMmB0SvNCNhh4YEJa3aaJ_OTG1eCL-wgim01QnZ21i8PuxpzMBHDX340aPNXgqmJJUcmWweFX3PppV59WD33U5ScsRiO3hUDV0pLb3EtZuEgBMmxND0Wl0Y9l035XOPc1Az5s0ean4sCu6kOc7FoNXNcMmzuvU-nudsOJrRVh_VFXy2O96Mw&state=Musf72BAN8p2PWes HTTP/1.1" 200 -


Access token obtained successfully.
Fetching all saved tracks...
Fetched 50 tracks so far...
Fetched 100 tracks so far...
Fetched 150 tracks so far...
Fetched 200 tracks so far...
Fetched 250 tracks so far...
Fetched 300 tracks so far...
Fetched 350 tracks so far...
Fetched 400 tracks so far...
Fetched 450 tracks so far...
Fetched 500 tracks so far...
Fetched 550 tracks so far...
Fetched 600 tracks so far...
Fetched 650 tracks so far...
Fetched 700 tracks so far...
Fetched 750 tracks so far...
Fetched 800 tracks so far...
Fetched 850 tracks so far...
Fetched 900 tracks so far...
Fetched 950 tracks so far...
Fetched 1000 tracks so far...
Fetched 1050 tracks so far...
Fetched 1100 tracks so far...
Fetched 1150 tracks so far...
Fetched 1200 tracks so far...
Fetched 1250 tracks so far...
Fetched 1300 tracks so far...
Fetched 1350 tracks so far...
Fetched 1400 tracks so far...
Fetched 1450 tracks so far...
Fetched 1500 tracks so far...
Fetched 1550 tracks so far...
Fetched 1600 tracks so fa

,rank,track_name,artist,album_name,duration_ms,explicit,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,1,F*ck Your Ethnicity,Kendrick Lamar,Section.80,224672,True,54,0.39200,0.601,0.883,0.000019,0,0.3970,-6.944,1,0.3280,150.967,0.5200
1,2,Not Like Us,Kendrick Lamar,Not Like Us,274192,True,88,0.01070,0.898,0.472,0.000000,1,0.1410,-7.001,1,0.0776,101.061,0.2140
2,3,"Everyday (feat. Rod Stewart, Miguel & Mark Ron...",A$AP Rocky,AT.LONG.LAST.A$AP,260987,True,78,0.00996,0.394,0.661,0.000000,4,0.3430,-8.684,1,0.1920,149.818,0.3350
3,4,"1Train (feat. Kendrick Lamar, Joey Bada$$, Yel...",A$AP Rocky,LONG.LIVE.A$AP (Deluxe Version),372173,True,61,0.35100,0.489,0.878,0.000000,2,0.6980,-3.360,1,0.3680,81.497,0.7750
4,5,HiiiPower,Kendrick Lamar,Section.80,279855,True,64,0.17900,0.695,0.827,0.000000,6,0.2830,-5.742,0,0.2820,81.024,0.5380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,2302,Dead by Daylight (Theme),Michel F. April,Dead by Daylight (Original Game Soundtrack),240841,False,43,0.03260,0.597,0.297,0.913000,5,0.1060,-12.158,0,0.0319,126.994,0.2540
2302,2303,Love Her Madly,The Doors,L.A. Woman,198467,False,67,0.05730,0.565,0.587,0.004930,4,0.0416,-7.393,0,0.0317,147.462,0.9660
2303,2304,Refur,Putumayo,Global Indie by Putumayo,209316,False,24,0.48300,0.481,0.344,0.002260,9,0.1130,-10.445,1,0.0268,103.206,0.2730
2304,2305,Tiny Dancer,Elton John,Madman Across The Water,377093,False,75,0.38200,0.414,0.428,0.000243,0,0.1480,-11.097,1,0.0278,145.075,0.2820
